In [1]:
import numpy as np
import pandas as pd
from cmath import phase

In [2]:
'''
Una función que a partir del nombre del fichero npy carge los datos, los redimensione y los devuelva. 
Al estilo de la función get_data(). Devuelve "datos_bruto". 

Los test de esta función:   comprobar que devuelve un array 2D, 
                            comprobar que la segunda dimensión tiene tamaño 291, 
                            comprobar que la matriz tiene una parte real 
                            y que tiene una parte imaginaria.
'''

def get_data(url):
    diccionario = np.load(url, allow_pickle=True).item()
    data = diccionario.get('sweep_data').get('data')
    data = data.reshape(data.shape[0],data.shape[2])
    return data

In [3]:
assert(len(get_data('Materiales\C01_V01.npy').shape) == 2) #comprobar que devuelve un array 2D
assert(get_data('Materiales\C01_V01.npy').shape[1] == 291) #comprobar que la segunda dimensión tiene tamaño 291
assert((get_data('Materiales\C01_V01.npy').real.mean) != None) #comprobar que la matriz tiene una parte real
assert((get_data('Materiales\C01_V01.npy').imag.mean) != None) #comprobar que la matriz tiene una parte imaginaria

In [4]:
'''
Una función get_modulo_fase() que a partir del array 2D anterior, obtenga un array 2D, con el doble de anchura. 
Por ejemplo, si "datos_bruto" es de 300x291, "modulo_fase" será de 600x291. 
Esa función obtendrá el módulo del número complejo y la fase del número complejo. 
    Módulo será una matriz de 300x291 y fase también. 
Será necesario concatenarlas "horizontalmente". 
Los test comprobarán las dimensiones y tamaños.
'''

def get_modulo_fase(data):
    modulo = abs(data)
    fase = []
    for i in data:
        fila = []
        for j in i:
            fila.append(phase(j)) #Fase
        fase.append(fila)
    fase = np.asarray(fase)
    modulo_fase = np.concatenate((modulo, fase), axis=0)
    return np.asarray(modulo_fase)
    

In [5]:
data = get_data('Materiales\C01_V01.npy')
assert(get_modulo_fase(data).shape[0] >= 600) #Dimensión de tamaño >= 600
assert(get_modulo_fase(data).shape[1] == 291) #Dimensión de tamaño = 291

In [59]:
'''
Una función que a partir de los datos "modulo_fase" devuelva la media. 
La comprobación sería que devuelva un array de 1 dimensión de tamaño 582 (291 x 2).
'''
def get_media(modulo_fase):
    traspuesta = np.transpose(modulo_fase) #Obtenemos la matriz traspuesta(291x600) de modulo_fase(600x291)
    
    #Separamos la traspuesta en modulo y fase
    t_modulo = traspuesta[:,:int(traspuesta.shape[1]/2)]
    t_fase = traspuesta[:,int(traspuesta.shape[1]/2):]
    
    media = []
    for i in t_modulo:
        media.append(i.mean())
    for j in t_fase:
        media.append(j.mean())
        
    return np.asarray(media)

In [60]:
modulo_fase = get_modulo_fase(data)
assert(get_media(modulo_fase).shape[0] == 582) #Dimensión de tamaño = 582
assert(len(get_media(modulo_fase).shape) == 1) #Array de 1 dimensión

In [61]:
'''
Función encargada de obtener las siguientes particiones separadas en entrenamiento (part_train) y test (part_test)

Partición 1
Entrenamiento: Carton Obj2-Obj10 + Plastico Obj2-Obj10 + Cristal Obj2-Obj10
Test: Carton Obj1 + Plastico Obj1 + Cristal Obj1

Partición 2
Entrenamiento: Carton Obj1,Obj3-Obj10 + Plastico Obj1,Obj3-Obj10 + Cristal Obj1,Obj3-Obj10
Test: Carton Obj2 + Plastico Obj2 + Cristal Obj2

...

Partición 10

TEST:
Sumando el número de filas de test1 y train1 de lo mismo que sumando el número de filas de test2 y train2.
Comprobar el tamaño de varias vistas igual a 582

'''

def particiones(materiales, objetos, vistas):
        
    part_train = [] #todos los entrenamientos
    part_test = [] #todos los test
    #part_train[0][0] se corresponde con part_test[0][0]

    for x in objetos: #Bucle para crear las 10 particiones
        train = [] #conjunto entrenamiento
        test = [] #conjunto de pruebas
        for y in materiales: #Bucle de los materiales = 3
            for i in objetos: #Bucle de los objetos = 10
                obj = [] #Lista de las vistas de un objeto
                for j in vistas: #Bucle de las vistas = 10
 
                    url = ('Materiales\%s%s_V%s.npy' % (y,i,j))
                    data = get_data(url)
                    modulo_fase = get_modulo_fase(data)
                    media = get_media(modulo_fase)
                    
                    obj.append(media)
                
                if x != i: #TRAIN
                    train = train + obj #train.append(obj)
                else: #TEST
                    test = test + obj
                    
        part_train.append(train) 
        part_test.append(test)


    part_train = np.asarray(part_train)
    part_test = np.asarray(part_test)

    return part_train, part_test

In [62]:
materiales = ['C','P','G'] #['C','P','G']
objetos = ['01','02','03','04','05','06','07','08','09','10'] #['01','02','03','04','05','06','07','08','09','10']
vistas = ['01','02','03','04','05','06','07','08','09','10'] #['01','02','03','04','05','06','07','08','09','10']

train, test = particiones(materiales, objetos, vistas)

In [63]:
len(train[0]), len(test[0]), len(train[1]), len(test[1]), len(train[9]), len(test[9])

(270, 30, 270, 30, 270, 30)

In [64]:
assert(len(train[0]) + len(test[0])) == (len(train[1]) + len(test[1])) #Train1 270 + Test1 30 = Train2 270 + Test2 30
assert(len(train[0]) + len(test[0])) == (len(train[9]) + len(test[9])) #Train1 270 + Test1 30 = Train10 270 + Test10 30
assert(len(train[0][0]) == 582) #La vista 1 del obj 1 es de tamaño 582 (291 x 2)(modulo x fase)
assert(len(train[0][0]) == len(train[0][9]) == len(test[5][7])) #Las vistas tinenen todas un tamaño de 582

In [65]:
np.save('train.npy', train)
np.save('test.npy', test)

In [ ]:
traindf = pd.DataFrame(train)
testdf = pd.DataFrame(train)